In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:

!pip install hd5y
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import tensorflow as tf
import pandas as pd
import numpy as np
import seaborn as sns
import re
import gc
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
# import keras
from tensorflow.keras.preprocessing.text import one_hot, Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding, Input, LSTM, Conv1D, MaxPool1D, Bidirectional, Dropout
from tensorflow.keras.models import Model
from keras.callbacks import ModelCheckpoint


ERROR: Could not find a version that satisfies the requirement hd5y (from versions: none)
ERROR: No matching distribution found for hd5y
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
f = open('/content/drive/My Drive/fnd-usingML/dataset/cleaned/train/text0.txt', 'r')
temp = f.read()
print(temp)

TOKYO (Reuters) - Japan is to acquire medium-range, air-launched cruise missiles,  capable of striking North Korea, a controversial purchase of what will become the longest-range munitions of a country that has renounced the right to wage war. Defence Minister Itsunori Onodera did not refer to North Korea when announcing the planned acquisition and said the new missiles would be for defence, with Japan still relying on the United States to strike any enemy bases.  We are planning to introduce the JSM (Joint Strike Missile)  that will be mounted on the F-35A (stealth fighter) as  stand-off  missiles that can be fired beyond the range of enemy threats,  Onodera told a news conference.  Japan is also looking to mount Lockheed Martin Corp s extended-range Joint Air-to-Surface Standoff Missile (JASSM-ER) on its F-15 fighters, he said. The JSM, designed by Norway s Kongsberg Defence & Aerospace , has a range of 500 km (310 miles). The JASSM-ER can hit targets 1,000 km away. The purchase plan

In [14]:
x_train = []
for i in range(0, 67508):
  f = open('/content/drive/My Drive/fnd-usingML/dataset/cleaned/train/text{}.txt'.format(i), 'r')
  temp = f.read()
  x_train.append(temp)
  f.close()

KeyboardInterrupt: ignored

In [13]:
x_test = []
for i in range(0, 67508):
  f = open('/content/drive/My Drive/fnd-usingML/dataset/cleaned/test/text{}.txt'.format{i}, 'r')
  temp = f.read()
  x_test.append(temp)
  f.close()

SyntaxError: ignored

In [ ]:
x_val = []
for i in range(0, 67508):
  f = open("/content/drive/My Drive/fnd-usingML/dataset/cleaned/val/text{}.txt".format{i}, 'r')
  temp = f.read()
  x_val.append(temp)
  f.close()

In [ ]:
y_train = []
f = open("/content/drive/My Drive/fnd-usingML/dataset/cleaned/train_label/labels.txt", 'r')
text = f.readlines()
for i in range(0,len(text)):
  temp = text[i].replace('\n','')
  # temp = float(temp)
  y_train.append(temp)
f.close()

In [ ]:
y_test = []
f = open("/content/drive/My Drive/fnd-usingML/dataset/cleaned/test_label/labels.txt", 'r')
text = f.readlines()
for i in range(0,len(text)):
  temp = text[i].replace('\n','')
  # temp = float(temp)
  y_test.append(temp)
f.close()

In [ ]:
y_val = []
f = open("/content/drive/My Drive/fnd-usingML/dataset/cleaned/val_label/labels.txt", 'r')
text = f.readlines()
for i in range(0,len(text)):
  temp = text[i].replace('\n','')
  # temp = float(temp)
  y_val.append(temp)
f.close()

In [ ]:
# Obtain additional stopwords from nltk
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [ ]:
# Remove stopwords and remove words with 2 or less characters
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3 and token not in stop_words:
            result.append(token)
            
    return result

In [ ]:
# Obtain the total words present in the dataset
list_of_words = []
for i in df_train.clean:
    for j in i:
        list_of_words.append(j)

In [ ]:
# Obtain the total number of unique words
total_words = len(list(set(list_of_words)))
total_words

In [ ]:
# join the words into a string
df_train['clean_joined'] = df_train['clean'].apply(lambda x: " ".join(x))

In [ ]:
# length of maximum document will be needed to create word embeddings 
maxlen = -1
for doc in df_train.clean_joined:
    tokens = nltk.word_tokenize(doc)
    if(maxlen<len(tokens)):
        maxlen = len(tokens)

In [ ]:
# Create a tokenizer to tokenize the words and create sequences of tokenized words
tokenizer = Tokenizer(num_words = total_words)
tokenizer.fit_on_texts(x_train)
train_sequences = tokenizer.texts_to_sequences(x_train)

In [ ]:
# Add padding can either be maxlen = 4406 or smaller number maxlen = 40 seems to work well based on results
padded_train = pad_sequences(train_sequences,maxlen = maxlen, padding = 'post', truncating = 'post') 

In [ ]:
# Sequential Model
model = Sequential()

# embeddidng layer
model.add(Embedding(183619 , output_dim = 128))
# model.add(Embedding(total_words, output_dim = 240))


# Bi-Directional RNN and LSTM
model.add(Bidirectional(LSTM(128)))

# Dense layers
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(1,activation= 'sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.summary()

In [ ]:
filepath="/content/drive/My Drive/fnd-usingML/checkpoints/title-and-text/weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, 
                             monitor = 'val_acc', 
                             verbose = 1, 
                             save_best_only = True, 
                             mode = 'max')

callbacks_list = [checkpoint]

In [ ]:
x_train = np.asarray(x_train)
x_test = np.asarray(x_test)
x_val = np.asarray(x_val)


y_train=np.asarray(y_train)
y_test=np.asarray(y_test)
y_val=np.asarray(y_val)


In [ ]:
for i in range(0,len(x_train)):
  np.fromstring(x_train[i], dtype=np.int8)
  # np.array(x_train[i])

for i in range(0,len(x_test)):
  # np.array(x_test[i])
  np.fromstring(x_test[i], dtype=np.int8)


for i in range(0,len(x_val)):
  # np.array(x_val[i])
  np.fromstring(x_val[i], dtype=np.int8)


In [ ]:
print(x_train[0].shape)
print(type(x_train[0]))

In [ ]:
# train the model
model.fit(x_train, y_train, batch_size = 64, validation_data=(x_val, y_val), epochs = 2, callbacks = callbacks_list)

In [ ]:
# make prediction
pred = model.predict(padded_test)

In [ ]:
# if the predicted value is >0.5 it is real else it is fake
prediction = []
for i in range(len(pred)):
    if pred[i].item() > 0.5:
        prediction.append(1)
    else:
        prediction.append(0)

In [ ]:
# getting the accuracy
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(list(y_test), prediction)

print("Model Accuracy : ", accuracy)

In [ ]:
# get the confusion matrix
# from sklearn.metrics import confusion_matrix
# cm = confusion_matrix(list(y_test), prediction)
# plt.figure(figsize = (25, 25))
# sns.heatmap(cm, annot = True)

In [ ]:
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

In [ ]:
text_test = 'The claim: Trump said, "Some Africans are lazy fools only good at eating, lovemaking and stealing" Falsely attributed quotes on social media that target politicians have become a popular form of misinformation, particularly as the 2020 election season enters its final stretch. A phrase claiming to come from President Donald Trump that circulated in October 2015 is making the rounds again on social media platforms. The article, allegedly published Oct. 27, 2015, claims Trump, while in Indianapolis, said African Americans are "lazy." The story does not reference a date or context for Trump speaking in that city. The social media post contains what appears to be a picture of the print version of the article, which has "Africans Are Lazy, Good At Sex, Theft", as a headline, above a photo of Trump. The article quotes Trump as referring to Africans and African Americans interchangeably. '
print(text_test)

In [ ]:
corpus = []

new = re.sub('[^a-zA-Z]', ' ', text_test)
new = new.lower()
new = new.split()
lemmatizer = WordNetLemmatizer() 
sw = stopwords.words('english')
new = [lemmatizer.lemmatize(word) for word in new if not word in set(sw)]
new = ' '.join(new)
corpus.append(new)

text_sequences_test = tokenizer.texts_to_sequences(corpus)
padded_text_test = pad_sequences(text_sequences_test,maxlen = 13682, truncating = 'post')
print(padded_text_test)


In [ ]:
prediction_text_test = model.predict(padded_text_test)
result = ''
# for i in range(len(prediction_text_test)):
if prediction_text_test[0] > 0.5:
    result = 'fake'
else:
    result = 'true'

print(result)
print(prediction_text_test)
